# SQL Subqueries

## Introduction

SQL queries can get complex. For example, you might have been a little thrown off from the many to many join in the last lab. There you had to join four tables. This is just the tip of the iceberg. Depending on how your database is set up, you might have to join subset views of multiple tables. When queries get complex like this, it is often useful to use the concept of subqueries to help break the problem into smaller, more digestible tasks.
M
## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## Our Customer Relation Managment Database Schema

As a handy reference, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.Connection('data.sqlite')
c = conn.cursor()

## All of the Employees From the United States

Let's start with a query of employees from the United States. Using your current knowledge, you could solve this using a join.

In [12]:
c.execute('select * from employees join offices using (officeCode) where country = "USA"')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
3,1143,Bow,Anthony,x5428,abow@classicmodelcars.com,1,1056,Sales Manager (NA),San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
4,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
5,1166,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1,1143,Sales Rep,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
6,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
7,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
8,1286,Tseng,Foon Yue,x2248,ftseng@classicmodelcars.com,3,1143,Sales Rep,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
9,1323,Vanauf,George,x4102,gvanauf@classicmodelcars.com,3,1143,Sales Rep,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA


In [13]:
c.execute("""select lastName, firstName, officeCode
                    from employees
                    join offices
                    using(officeCode)
                    where country = "USA";
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
len(df)

10

Another approach would be to use a subquery. Here's what it would look like.

In [16]:
c.execute('''select * from employees where officeCode
                    in (select officeCode from offices where country = "USA")''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1143,Bow,Anthony,x5428,abow@classicmodelcars.com,1,1056,Sales Manager (NA)
4,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep
5,1166,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1,1143,Sales Rep
6,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep
7,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep
8,1286,Tseng,Foon Yue,x2248,ftseng@classicmodelcars.com,3,1143,Sales Rep
9,1323,Vanauf,George,x4102,gvanauf@classicmodelcars.com,3,1143,Sales Rep


In [4]:
c.execute("""select lastName, firstName, officeCode
                    from employees
                    where officeCode in (select officeCode from offices where country = "USA");
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,lastName,firstName,officeCode
0,Murphy,Diane,1
1,Patterson,Mary,1
2,Firrelli,Jeff,1
3,Bow,Anthony,1
4,Jennings,Leslie,1
5,Thompson,Leslie,1
6,Firrelli,Julie,2
7,Patterson,Steve,2
8,Tseng,Foon Yue,3
9,Vanauf,George,3


There it is! A query within a query! This can be very helpful and also allow you to break down problems into constituent parts. Often queries can be formulated in multiple ways as with the above example. Other times, using a subquery might be essential. For example, what if you wanted to find all of the employees from offices with at least 5 employees?  

Think for a minute about how you might write such a query.  


Now that you've had a minute to think it over, you might see some of the challenges with this query. On the one hand, we are looking to filter based on an aggregate condition: the number of employees per office. You know how to do this using a groupby and having clause, but the data we wish to retrieve is not aggregate data: we only wish to filter based on the aggregate, not retrieve aggregate data. As such, this is a natural place to use a subquery.

In [17]:
c.execute("""select lastName, firstName, officeCode from employees
                    where officeCode in (select officeCode 
                                                from offices 
                                                join employees
                                                using(officeCode)
                                                group by 1
                                                having count(employeeNumber) >= 5
                                         );
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,lastName,firstName,officeCode
0,Murphy,Diane,1
1,Patterson,Mary,1
2,Firrelli,Jeff,1
3,Bondur,Gerard,4
4,Bow,Anthony,1
5,Jennings,Leslie,1
6,Thompson,Leslie,1
7,Bondur,Loui,4
8,Hernandez,Gerard,4
9,Castillo,Pamela,4


You can chain queries like this in many fashions. For example, maybe you're also a statistical geek and want to find the average of individual customers average payments:

(It might be more interesting to investigate the standard deviation of customer's average payments, but standard deviation is not natively supported in sqlite as it is in other sql versions like postgreSQL.)

In [18]:
c.execute("""select AVG(avgPayment)
                    from (select AVG(amount) as avgPayment from payments);
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,AVG(avgPayment)
0,32431.645531


## Summary

In this lesson, you were briefly introduced to the powerful concept of subqueries and how you can use them to write more complex queries. In the upcoming lab you'll really start to strengthen your SQL and data wrangling skills by using all of the SQL techniques introduced thus far.